The goal of our project was to see the Boston housing data could reliably predict the median house price utilizing machine learning and Python. We also wanted to see if filtering out different values and inputs could help to improve the model’s efficiency and performance, and if we could manipulate the data to provide more accurate estimates.

In [ ]:
# Dependencies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

# Keras specific
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.linear_model import LinearRegression

Loading and Cleaning Data

First we had to import the data in our Jupyter Notebook, then we went about cleaning it. First we renamed the columns in our data set to be more intuitive, making sure we could read what we were looking at without confusion. Then we checked to see if any rows had null/blank values, which we did find several of. 

In [ ]:
# Read csv using pandas. We should specify the column index to avoid mislabelling problems when data is uploaded
data=pd.read_csv('data/housingdata.csv',index_col=False)

In [ ]:
# data.head to check the first few columns of the data set
data.head()

In [ ]:
# We rename the columns now using data.columns function
data.columns=['Crime Rate','Residential Land Zone','Non-retail business acres','Charles River Variable','Nitric Oxide Concentration',
             'Rooms','Age','Distance','Accessiblity to Highway','Tax Rate','Pupil-Teacher ratio','Proportion of Blacks','% lower status',
             'Median Value']

In [ ]:
# data.head after renaming the columns
data.head()

In [ ]:
# let's drop proportion of blacks
# We use data.drop to drop the proportion of blacks
data.drop("Proportion of Blacks",axis=1, inplace=True)

In [ ]:
# data.head to verify
data.head()

In [ ]:
# we use describe for sample stats and central tendency stats
data.describe()

In [ ]:
# We use data.info to get the data types and count of non-nulls in the dataset
data.info()

In [ ]:
# From the result above, we found out that there is a possibility of nulls or missing data which we can count by summing all nulls
# checking for nulls
data.isnull().any()

At this point we had two thoughts on how to handle the null values. One was to simply drop the rows with null values and continue, and the other was imputation. Meaning adding in the median value of a given column in place of each null value. After discussing we decided to run the model we were building on both methods and see which performed better.

In [ ]:
data_dropna = data.copy()
data_dropna = data_dropna.dropna()
#data.info()


In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
data.iloc[:,:]=imputer.fit_transform(data)
data.info()

In [ ]:
# Checking for nulls or missing values
data.isnull().any()

In [ ]:
# Counting nulls or missing values
data.isnull().sum()

Analysis

Once we had cleaned up the data using both methodologies, we moved on to some early analysis. One of the first things we did was use a Seaborn Heatmap to see which metrics had high correlation with media house value.

In [ ]:
# Now, let's check for a correlation between the target value, which is our Median House price values, and all the other columns.
# Let's check then for a correlation to our median value (which of the variables are highly correlated to it?)
# We use seaborn heatmap. We should also consider the effects of outliers as well. But, let's check for correlation first
# Using seaborn
plt.figure(figsize=(18, 18))
sns.heatmap(data.corr(),annot = True,cmap= 'coolwarm', linewidths=1, linecolor='white',fmt='.2g')
plt.savefig("output/Seaborn.png")

Looking at the last column, we found out that the # of rooms (0.7) and % lower status (-0.74) have a strong correlation to the median house value. This means they directly influence the price. The correlation analysis shows that median house value is highly correlated to % lower status and the number of rooms per dwelling.
The total number of Rooms is positively correlated to Median Value. So as number of Rooms increases, the Median value increases. The opposite is true for % lower status: when % lower status goes up, the price goes down.

Stephs Code Here

# Export Dataframe to CSV

In [ ]:
data.to_csv(r'data\revisedhousingdata.csv', index = False)

Once we had the correlation, we prepared several scatter plots to take a look at the highly correlated metrics compared against the house value in order to see how the data compared overall. 

In [ ]:
data.head()
data['Home Value'] = (data['Median Value'] * 1000)
data.head()

In [ ]:
# Generate a scatter plot of Charles River Variable(distance from the river) vs. value
crimeRate = data.iloc[:,0]
value = data.iloc[:,13]

plt.scatter(crimeRate, value, color = '#545E45')

#label
plt.title('Crime Rate vs Value')
plt.xlabel('Crime Rate')
plt.ylabel('Value')
plt.show()

In [ ]:
# Generate a scatter plot of Charles River Variable(distance from the river) vs. value
charlesRiver = data.iloc[:,3]
value = data.iloc[:,13]

plt.scatter(charlesRiver, value, color = '#8D2B00')

#label
plt.title('Distance from Charles River vs Value')
plt.xlabel('Distance')
plt.ylabel('Value')
plt.show()

In [ ]:
# Generate a scatter plot of Nitric Oxide Concentration vs. value
nitricOxide = data.iloc[:,4]
value = data.iloc[:,13]

plt.scatter(nitricOxide, value, color = '#BE6731')

#label
plt.title('Nitric Oxide vs Value')
plt.xlabel('Nitric')
plt.ylabel('Value')
plt.savefig("output/NitricOxideVsValue.png")
plt.show()

In [ ]:
# Generate a scatter plot of rooms vs. value
rooms = data.iloc[:,5]
value = data.iloc[:,13]

plt.scatter(rooms, value, color = '#76704C')


#label
plt.title('Number of Rooms vs Value')
plt.xlabel('Rooms')
plt.ylabel('Value')
plt.savefig("output/NumberOfRoomsVsValue.png")
plt.show()

In [ ]:
# Generate a scatter plot of age vs. value
age = data.iloc[:,6]
value = data.iloc[:,13]

plt.scatter(age, value, color = '#B55119')

#label
plt.title('Age of Home vs Value')
plt.xlabel('Age')
plt.ylabel('Value')
plt.savefig("output/AgeVsValue.png")
plt.show()

In [ ]:
# Generate a scatter plot of % lower status vs. value
lowerStatus = data.iloc[:,11]
value = data.iloc[:,13]

plt.scatter(lowerStatus, value, color = '#1F2D16')

#label
plt.title('% Lower Status vs Value')
plt.xlabel('Status')
plt.ylabel('Value')
plt.savefig("output/StatusVsValue.png")
plt.show()

Building The Model

After our exploratory analysis we started to work on our model. After discussing our data with our instructor Dom, we went with a multiple linear regression neural network using MSE (Mean Squared Error) and R Squared Score to rate the models efficacy at his suggestion.

After researching how to modify a neural network to fit our needs, we went with a sequential model using relu activation for the analysis and had a single output node using linear activation. As suggested, we had the model use mean squared error as the primary output in order to help us troubleshoot and get a rough gauge of the models performance.

It was at this point we realized we had not yet standardized our data, which is a general best practice due to the different ways our data is being measured which makes comparison difficult. Instead of just standardizing it we decided to compare pre/post standardization model performance in order to quantify the level of impact that standardization would have.

Initial Model Before Standardizing Data

In [ ]:
# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt

# Keras specific
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# Assign the data to X and y
X = data[['Crime Rate','Residential Land Zone','Non-retail business acres','Charles River Variable','Nitric Oxide Concentration',
             'Rooms','Age','Distance','Accessiblity to Highway','Tax Rate','Pupil-Teacher ratio','% lower status']]
y = data['Median Value'].values.reshape(-1,1)
print(X.shape, y.shape)

In [ ]:
# Use train_test_split to create training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=150)
print(X_train.shape)
print(X_test.shape)

In [ ]:
#Creating the model
model = Sequential()
model.add(Dense(36, input_dim=12, activation= "relu"))
model.add(Dense(1, activation='linear'))

In [ ]:
model.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model.fit(X_train, y_train, epochs=100)

In [ ]:
# Make a test preciction to see if we're on the right track
row = 3
test_row = X_test.iloc[row, :]
test_row_array = np.array(test_row).reshape(1, 12)
test_row_array.shape

print(f'Prediction = {model.predict(test_row_array)}')
print(f'Actual = {y_test[row]}')

In [ ]:
#Use X_test to predict y, and use that to calculate our R score
y_test_predict = model.predict(X_test)
r2_score(y_test, y_test_predict)

In [ ]:
model.evaluate(X_test,y_test, verbose=2)

The R2 score for our model is not very reliable at a .37 which would put it in the higher of the of low reliability range for a model. Next we will standarize our data and then see if that improves our R score.

Model With Standardized Data

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

In [ ]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [ ]:
model_scaled = Sequential()
model_scaled.add(Dense(36, input_dim=12, activation= "relu"))
model_scaled.add(Dense(1, activation='linear'))

model_scaled.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_scaled.fit(X_train_scaled, y_train_scaled, epochs=100)

#r2_score(y_train_scaled, y_test_scaled)

In [ ]:
#Use X_test to predict y, and use that to calculate our R score
y_test_scaled_predict = model_scaled.predict(X_test_scaled)
r2_score(y_test_scaled, y_test_scaled_predict)

In [ ]:
#Evaluate the model
model_scaled.evaluate(X_test_scaled,y_test_scaled, verbose=2)

There was a stark difference between the two models, showing that standardizing the data clearly improved performance of the model with the MSE decreasing considerably, and the R2 score improving majorly.

Tinkering with the Data

Once we standardized the data we took a look to see if removing some columns of data would help to improve the performance of the model. Looking at our Seaborn Heatmap we saw that both the Charles River and Distance inputs has very low correlation to house value. We dropped those two columns from our data, and created a new model using the same criteria to measure the overall impact on performance.


In [ ]:
manipulated_data = data.copy()

manipulated_data = manipulated_data.drop(["Charles River Variable", 'Distance'],axis=1)
manipulated_data.head()

In [ ]:
X_manipulated = manipulated_data[['Crime Rate','Residential Land Zone','Non-retail business acres','Nitric Oxide Concentration',
             'Rooms','Age','Accessiblity to Highway','Tax Rate','Pupil-Teacher ratio','% lower status']]
y_manipulated = manipulated_data['Median Value'].values.reshape(-1,1)
#print(X.shape, y.shape)

X_train_manipulated, X_test_manipulated, y_train_manipulated, y_test_manipulated = train_test_split(X_manipulated, y_manipulated, random_state=150)
#print(X_train.shape)
#print(X_test.shape)

In [ ]:
X_scaler_manipulated = StandardScaler().fit(X_train_manipulated)
y_scaler_manipulated = StandardScaler().fit(y_train_manipulated)

X_train_scaled_manipulated = X_scaler_manipulated.transform(X_train_manipulated)
X_test_scaled_manipulated = X_scaler_manipulated.transform(X_test_manipulated)
y_train_scaled_manipulated = y_scaler_manipulated.transform(y_train_manipulated)
y_test_scaled_manipulated = y_scaler_manipulated.transform(y_test_manipulated)

model_manipulated = Sequential()
model_manipulated.add(Dense(30, input_dim=10, activation= "relu"))

model_manipulated.add(Dense(1, activation='linear'))

model_manipulated.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_manipulated.fit(X_train_scaled_manipulated, y_train_scaled_manipulated, epochs=100)

model_manipulated.evaluate(X_test_scaled_manipulated,y_test_scaled_manipulated, verbose=2)

In [ ]:
#Use X_test to predict y, and use that to calculate our R score
y_test_scaled_manip_predict = model_manipulated.predict(X_test_scaled_manipulated)
r2_score(y_test_scaled_manipulated, y_test_scaled_manip_predict)

We found removing those two columns did have a mild impact on performance, however upon repeated running of the model we found that the results would change seemingly at random. This led us to believe that removing the two columns was not meaningfully different.

Next we tried selected the top 6 columns based on correlation to see if hand selecting the data would be more beneficial. We went Non-retail business acres, Nitric Oxide Concentration, Rooms, Tax Rate, Pupil-Teacher ratio, % lower status as our columns.

In [ ]:
X_cherrypicked = data[['Non-retail business acres','Nitric Oxide Concentration',
             'Rooms','Tax Rate','Pupil-Teacher ratio','% lower status']]
y_cherrypicked = data['Median Value'].values.reshape(-1,1)
#print(X.shape, y.shape)

X_train_cherrypicked, X_test_cherrypicked, y_train_cherrypicked, y_test_cherrypicked = train_test_split(X_cherrypicked, y_cherrypicked, random_state=150)
#print(X_train.shape)
#print(X_test.shape)

In [ ]:
X_scaler_cherrypicked = StandardScaler().fit(X_train_cherrypicked)
y_scaler_cherrypicked = StandardScaler().fit(y_train_cherrypicked)

X_train_scaled_cherrypicked = X_scaler_cherrypicked.transform(X_train_cherrypicked)
X_test_scaled_cherrypicked = X_scaler_cherrypicked.transform(X_test_cherrypicked)
y_train_scaled_cherrypicked = y_scaler_cherrypicked.transform(y_train_cherrypicked)
y_test_scaled_cherrypicked = y_scaler_cherrypicked.transform(y_test_cherrypicked)

model_cherrypicked = Sequential()
model_cherrypicked.add(Dense(18, input_dim=6, activation= "relu"))
model_cherrypicked.add(Dense(1, activation='linear'))

model_cherrypicked.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_cherrypicked.fit(X_train_scaled_cherrypicked, y_train_scaled_cherrypicked, epochs=100)

model_cherrypicked.evaluate(X_test_scaled_cherrypicked,y_test_scaled_cherrypicked, verbose=2)

In [ ]:
#Use X_test to predict y, and use that to calculate our R score
y_test_scaled_cherrypicked_predict = model_cherrypicked.predict(X_test_scaled_cherrypicked)
r2_score(y_test_scaled_cherrypicked, y_test_scaled_cherrypicked_predict)

Again, we saw the results of the test change upon running them, with only slight differences in performance. After seeing similar changes to simply dropping the two columns, it seemed wiser to keep the data set as is, in order to preserve the integrity of our data set.
Next we decided to compare the data set cleaning methods we used. 

Below is a model using a data set where we instead drop the rows contianing nulls.

In [ ]:
data_dropna

In [ ]:
X_drop = data_dropna[['Crime Rate','Residential Land Zone','Non-retail business acres','Charles River Variable','Nitric Oxide Concentration',
             'Rooms','Age','Distance','Accessiblity to Highway','Tax Rate','Pupil-Teacher ratio','% lower status']]
y_drop = data_dropna['Median Value'].values.reshape(-1,1)

X_train_drop, X_test_drop, y_train_drop, y_test_drop = train_test_split(X_drop, y_drop, random_state=150)

X_scaler_drop = StandardScaler().fit(X_train_drop)
y_scaler_drop = StandardScaler().fit(y_train_drop)

X_train_scaled_drop = X_scaler_drop.transform(X_train_drop)
X_test_scaled_drop = X_scaler_drop.transform(X_test_drop)
y_train_scaled_drop = y_scaler_drop.transform(y_train_drop)
y_test_scaled_drop = y_scaler_drop.transform(y_test_drop)

model_drop = Sequential()
model_drop.add(Dense(36, input_dim=12, activation= "relu"))
model_drop.add(Dense(1, activation='linear'))

model_drop.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_drop.fit(X_train_scaled_drop, y_train_scaled_drop, epochs=100)

model_drop.evaluate(X_test_scaled_drop,y_test_scaled_drop, verbose=2)

In [ ]:
y_test_scaled_predict_drop = model_drop.predict(X_test_scaled_drop)
r2_score(y_test_scaled_drop, y_test_scaled_predict_drop)

We had assumed that providing additional inputs using the imputation method would actually improve the models performance, but we found that wasn’t the case at all. The data set where we had dropped all null values had a much higher R2 score and a lower MSE. 

Lastly we wanted to compare what the performance looked like if we removed the outliers from the dataset. For this analysis we used the drop NA data since it was more accurate.

In [ ]:
from scipy import stats

data_remove_outliers = data_dropna.copy()
data_remove_outliers = data_remove_outliers[(np.abs(stats.zscore(data_remove_outliers)) < 3).all(axis=1)]
data_remove_outliers

In [ ]:
X_outlier = data_remove_outliers[['Crime Rate','Residential Land Zone','Non-retail business acres','Charles River Variable','Nitric Oxide Concentration',
             'Rooms','Age','Distance','Accessiblity to Highway','Tax Rate','Pupil-Teacher ratio','% lower status']]
y_outlier = data_remove_outliers['Median Value'].values.reshape(-1,1)

X_train_outlier, X_test_outlier, y_train_outlier, y_test_outlier = train_test_split(X_outlier, y_outlier, random_state=150)

X_scaler_outlier = StandardScaler().fit(X_train_outlier)
y_scaler_outlier = StandardScaler().fit(y_train_outlier)

X_train_scaled_outlier = X_scaler_outlier.transform(X_train_outlier)
X_test_scaled_outlier = X_scaler_outlier.transform(X_test_outlier)
y_train_scaled_outlier = y_scaler_outlier.transform(y_train_outlier)
y_test_scaled_outlier = y_scaler_outlier.transform(y_test_outlier)

model_outlier = Sequential()
model_outlier.add(Dense(36, input_dim=12, activation= "relu"))
model_outlier.add(Dense(1, activation='linear'))

model_outlier.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])
model_outlier.fit(X_train_scaled_outlier, y_train_scaled_outlier, epochs=100)

model_drop.evaluate(X_test_scaled_drop,y_test_scaled_drop, verbose=2)

In [ ]:
y_test_scaled_predict_outlier = model_outlier.predict(X_test_scaled_outlier)
r2_score(y_test_scaled_outlier, y_test_scaled_predict_outlier)


After which we saw that removing outliers did not significantly move the needly on either the MSE or R2 score. 

Conclusion.

With our model getting a MSE of 0.10 and a R2 score of 0.90 we found that the model could be used to somewhat accurately predict the value of a house based on the provided inputs. We also found that our attempts at manipulating the data proved ultimately proved counter productive as each time we manipulated the data it either had no change, or hurt performance of our model. With the only exception being the standardization of the model.
